# PC2135: Thermodynamics and Statistical Physics - Mini Project I

### Chin Jia Yao, Yoo Jie Lerk

## Setup

We consider 5 ideal gas particles constrained within a 3D box of dimensions $L \times L \times L$, having energy $200E_0$, where

$$
\begin{equation}
    E_0 = \frac{\hbar^2}{2m}\frac{\pi}{L^2}
\end{equation}
$$

Since the particles exist in a 3D box, the Schrödinger equation results in a wave equation with 3 degrees of freedom &mdash; 3 $n$'s required to completely specify the state of each particle. From there, we can get that the energy of the $i$-th particle will be

$$
\begin{equation}
\tag{2}
    E_i = (n_{x_i}^2 + n_{y_i}^2 + n_{z_i}^2)E_0
\end{equation}
$$

Hence, summing over the energies of all five particles, we must get $200E_0$ which implies

$$
\begin{equation}
\tag{3}
    \sum_{i=1}^5 (n_{x_i}^2 + n_{y_i}^2 + n_{z_i}^2) = 200
\end{equation}
$$

By finding the set of all 15-tuple of $n$'s that satisfy to equation (3), we will then be able to know the full set of microstates that result in the measured macroscopic energy level (i.e. all _accessible_ states). Only then can we perform further calculations on the probability of states.

Our approach hence first focuses on solving the above equation.

## Enumerating unique accessible states

To solve equation (3) which has 15 unknowns, our approach is simple. We first find the unique configurations (unique under permutation) by finding the $n$'s in increasing order.

1. Fix the number of the first $n$, with maximum given such that $n_{max}^2 < 200$
2. Subtract $n^2$ from the energy threshold: $200-n^2$
3. Recursively solve for the remaining degrees of freedom, updating the threshold for each call.
4. Each recursive call only considers $n_{min}$ that is larger or equal to last found $n$.

However, we will need to filter for _impossible_ configurations, which means that when the recursion reaches the last degree of freedom, there exists no positive integer $n$ that fulfills $n^2 = E_{threshold}$.

Thus, we will need to filter for these microstates, and this can be done by simply checking whether the last $E_{threshold}$ (i.e. energy remaining for the last degree of freedom) is a perfect square.
- If yes, then we include that configuration into the accesible states
- If no, then we put the last $n$ as a 0 to be filtered afterwards

All the valid configurations will then be added to a list, where the function `get_microstates` will then return this list of all accessible states. 

\
`get_microstates` takes in the _number of particles_ and calls the helper function `get_configuration` to transform each particle into 3 degrees of freedom.

In [8]:
import numpy as np
import math

dimension = 3
E_total = 200 # multiples of E0
n_particles = 5  # number of particles


def get_unique_microstates(n_E: int, num_particles: int):
    states = get_configuration(n_E, num_particles * dimension)
    mask = np.where(states[:,0] != 0)

    valid_states = states[mask]
    return np.flip(valid_states, axis = 1)

def get_configuration(n_E: int, degrees: int, last_n: int = 1):
    max_n = int(np.ceil(np.sqrt(n_E)))
    microstates = np.empty(shape=[0, degrees])

    is_square = (max_n * max_n == n_E)

    if degrees==1:
        if is_square and max_n>=last_n:
            return np.array([[max_n]])
        else:
            return np.array([[0]])

    for i in range(last_n, max_n):
        sub_states = get_configuration(n_E - i**2, degrees - 1, i)
        row_num = len(sub_states)

        # add n=i to all currently found substates
        appendage = np.empty(shape=[row_num,1]) 
        appendage.fill(i)

        sub_states = np.append(sub_states, appendage, axis = 1)

        for state in sub_states:
            microstates = np.append(microstates, [state], axis = 0)

    return microstates

states = get_unique_microstates(E_total, n_particles)
print("Unique unpermutated states:\n")
print(states)

Unique unpermutated states:

[[ 1.  1.  1. ...  2.  9. 10.]
 [ 1.  1.  1. ...  4.  5. 12.]
 [ 1.  1.  1. ...  6.  7. 10.]
 ...
 [ 2.  3.  3. ...  4.  5.  7.]
 [ 2.  3.  3. ...  4.  4.  4.]
 [ 3.  3.  3. ...  4.  4.  5.]]


# Enumerating permutations of unique states

Since we obtained all unique configurations, here we introduce 2 helper functions to help us in calculating the total number of microstates.

<br>

1. `get_number_permutations` returns the number of permutations for a _specific_ state through the permutation formula $\frac{15!}{c_1! c_2!...}$


2. `get_total_permutations` returns the total number of permutations for a _set_ of unique microstates

In [9]:
def get_number_permutations(specific_microstate: np.ndarray):
    uniques, count = np.unique(specific_microstate, axis=0, return_counts=True)
    l = len(specific_microstate)

    ans = math.factorial(l)
    for i in count:
        ans = ans / math.factorial(i)

    return ans


def get_total_permutations(microstates: np.ndarray):
    counter = 0
    for state in microstates:
        counter += get_number_permutations(state)

    return counter

print("Total number of states:")
print(get_total_permutations(states))

Total number of states:
14543054580.0


## (a) Finding minimum energy and its states

Next, we evaluate the single particle states to determine the state with the minimum energy. Note that the 15 numbers in each microstate can be randomly grouped into groups of three to indicate the configuration of each particle.

Since the elements in each of the unique microstates are already in ascending order, we only need to consider the first 3 values in each configuration to find the particle with minimum energy in each microstate.

In [15]:
def find_min(microstates: np.ndarray):
    first_3 = microstates[:, :dimension]
    sums = [np.sum(arr * arr) for arr in first_3]
    min = np.min(sums)
    mins = np.where(sums == min)
    uniques = np.unique(first_3[mins], axis=0)
    return (uniques, min)

min, min_e = find_min(states)
print("Possible state(s) of particle with minimum possible energy:")
print(min)
print("Minimum energy in terms of E0:")
print(min_e)

Possible state(s) of particle with minimum possible energy:
[[1. 1. 1.]]
Minimum energy in terms of E0:
3.0


## Counting states with first particle having minimum energy

Having obtained the possible states with minimum energy, we search through all the unique states to find states with at least one particle in the minimum energy configuration. 

For each such state, we can count the number of permutations available for the remaining $12$ $n$'s. This multiplied by the number of permutations available for the first particle gives the total permutations for that specific microstate.

The sum of these permutations then gives the total number of states with the first particle having the minimum possible energy.

In [16]:
def enumerate_min(microstates: np.ndarray):
    mins = find_min(microstates)[0]
    counter = 0

    for min in mins:
        min_mask = np.where((microstates[:, :dimension] == min).all(axis=1), True, False)
        min_states = microstates[min_mask]
        counter += get_total_permutations(min_states[:, dimension:]) * get_total_permutations([min])
        
    return counter

print(f"Number of states with first particle having minimum energy {min_e} E0:")
print(enumerate_min(states))

Number of states with first particle having minimum energy 3.0 E0:
169580664.0


## (c) Finding maximum energy and its states

Similarly, we evaluate the single particle states to determine the states with the maximum energy. Again since the 15 numbers can be randomly grouped into 3, we simply group the largest 3 numbers and designate it as the first particle to find the maximum energy.

Since the elements in each of the unique microstates are already in ascending order, we only need to consider the last 3 values in each configuration.

In [18]:
def find_max(microstates: np.ndarray):
    last_3 = microstates[:, -dimension:]
    sums = [np.sum(arr * arr) for arr in last_3]
    max = np.max(sums)
    maxes = np.where(sums == max)
    uniques = np.unique(last_3[maxes], axis = 0)
    return (uniques, max)

max, max_e = find_max(states)
print("Possible state(s) of particle with maximum possible energy:")
print(max)
print("Maximum energy in terms of E0:")
print(max_e)

Possible state(s) of particle with maximum possible energy:
[[ 2.  9. 10.]
 [ 4.  5. 12.]
 [ 6.  7. 10.]]
Maximum energy in terms of E0:
185.0


## Counting states with first particle having maximum energy

Similarly, having obtained the possible states with maximum energy, we search through all the unique states to find states with at least one particle in the minimum energy configuration. 

For each such state, we can count the number of permutations available for the remaining $12$ $n$'s and multiply it by the number of permutations available for the first particle.

The sum of these permutations then gives the total number of states with the first particle having the maximum possible energy.

In [19]:
def enumerate_max(microstates: np.ndarray):
    maxes = find_max(microstates)[0]
    counter = 0

    for max in maxes:
        max_mask = np.where((microstates[:, -dimension:] == max).all(axis=1), True, False)
        max_states = microstates[max_mask]

        counter += get_total_permutations(max_states[:, :-dimension]) * get_total_permutations([max])

    return counter

print(f"Number of states with first particle having maximum energy {max_e} E0:")
print(enumerate_max(states))

Number of states with first particle having maximum energy 185.0 E0:
216.0


## (b), (d) Calculating probabilities

Since all microstates are equally probable, we can then obtain the probability of the first particle having minimum energy by dividing the number of such microstates that fulfill this condition by the total number of available microstates. 

The probability of the first particle having maximum energy is calculated in a similar manner.

In [21]:
def calc_prob_min_energy(microstates: np.ndarray):
    return enumerate_min(microstates) / get_total_permutations(states)


def calc_prob_max_energy(microstates: np.ndarray):
    return enumerate_max(microstates) / get_total_permutations(states)

print(f"Probabity of first particle having minimum energy {min_e} E0:")
print(calc_prob_min_energy(states))

print(f"\nProbabity of first particle having maximum energy {max_e} E0:")
print(calc_prob_max_energy(states))

Probabity of first particle having minimum energy 3.0 E0:
0.011660594620418456

Probabity of first particle having maximum energy 185.0 E0:
1.4852450619077577e-08


## Final answers:
(a) the lowest possible energy (in terms of $E_0$) possible for the first particle. <br>

$ 3E_0 $

<br>

(b) the probability of that particle having said energy. <br>

$ 0.01166 $

<br>

(c) the highest energy (in terms of $E_0$) possible for the first particle. <br>

$ 185E_0 $

<br>

(d) the probability of that particle having said energy. <br>

$ 1.4852 \times 10^{-8} $

## Discussion

#### From the fundamental postulate of equilibrium statistical mechanics, all accessible energy levels should be equally probable for the particle. Explain how your results agrees or disagrees with the fundamental postulate.

In our final calculated answer, the probability of the first particle having minimum energy is much larger than the probability of it having maximum energy. Even so, this result does **not** disagree with the fundamental postulate. There are two ways to explain this.

<br>

The fundamental postulate states that for an _isolated system in equilibrium_, it can be found in each of its accessible states with equal probability. Hence

1. The initial condition is not true in this exercise, since the energy of the first particle is not isolated. The macroscopic energy is known to be $200E_0$, which means the energy of the first particle also depends on the configuration of all other particles. Hence the postulate is not applicable here.

2. The fundamental postulate mentions each state, not the energy of a particle. As shown in (c), the same energy value can be produced from different unique configurations.

Thus the fundamental postulate is not violated here, for the two reasons stated above.